# Q&A Chatbot using LangChain + FAISS + Ollama

This notebook implements a PDF‑based Question‑Answering chatbot using:
- **LangChain**
- **FAISS vector store**
- **Ollama LLM**
- **Embeddings for semantic search**

---

## 📁 Project Structure
- Load PDF documents  
- Split and embed  
- Store embeddings in FAISS  
- Build a retrieval pipeline  
- Generate answers using Ollama  

---

## ⚙️ Setup
Install dependencies listed in `requirements.txt`.


In [ ]:
!pip install langchain ollama openai faiss-cpu chromadb

In [ ]:
import os
from dotenv import load_dotenv

In [ ]:
MODEL = "phi"

In [ ]:
!pip install langchain_community

In [ ]:
!pip install -U langchain-ollama


In [ ]:
from langchain_ollama import OllamaLLM, OllamaEmbeddings

model = OllamaLLM(model=MODEL)

embeddings = OllamaEmbeddings(model=MODEL)

In [ ]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [ ]:
from langchain_core.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't
answer the question, reply "say i don't know"

context: {context}

question: {question}
"""
prompt = PromptTemplate.from_template(template)

In [ ]:
!pip install pypdf

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(r"C:\Users\ASUS\Desktop\LLMs\Q&A Chatbot\jesc103.pdf")

pages = loader.load_and_split()

In [ ]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(
    pages,
    embedding = embeddings
)

In [ ]:
retriever = vectorstore.as_retriever()

In [ ]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question")
    }
    | prompt
    | model
    | parser
)

# chain.invoke({"question":"Why do ionic compounds have high melting points?"})

In [ ]:
def ask_question(q, chain):
    result = chain.invoke({'question': q})
    return result

In [ ]:
while True:
    q = input('Your question: ')
    if q.lower() in 'exit quit bye':
        print('Bye bye!')
        break
    result = ask_question(q, chain)
    print(result)
    print('-' * 100)